## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/days/day02/solution/day02_solution.ipynb)

# ⚡ Day 2 – Fossil Fuels vs. Renewables
### Comparing energy transitions with grouped bar charts

We now move from climate outcomes to energy systems. Today's notebook helps you quantify how five major economies are shifting their energy mix — and how different renewable technologies contribute. We'll work in short cycles again: load → check → transform → visualize → reflect.

### Data card — Our World in Data energy share series
- **Source:** [Our World in Data – Energy Mix](https://ourworldindata.org/energy-mix)
- **Temporal coverage:** 1965–2023 (annual)
- **Geographic coverage:** Global plus individual countries/regions
- **Units:** Share of primary energy consumption (% of total)
- **Files used today:** Total renewable share, hydro share, solar share, wind share
- **Processing notes:** Shares are calculated from BP Statistical Review. Negative or missing values can appear for smaller countries.
- **Caveats:** Country definitions follow BP conventions (e.g., EU aggregated). Shares may not sum to 100% because non-renewable categories are not included in this subset.

### Preview: today's target chart
A grouped bar chart comparing renewable technology shares in 2023, with a caption that spells out the headline claim.

In [ ]:
from pathlib import Path
from warnings import warn

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

CANDIDATES = [Path.cwd(), *Path.cwd().parents]
for candidate in CANDIDATES:
    if (candidate / "data").exists():
        PROJECT_ROOT = candidate
        break
else:
    raise FileNotFoundError("Couldn't find the project root containing a data/ folder.")

DATA_DIR = PROJECT_ROOT / "data"
PLOTS_DIR = PROJECT_ROOT / "plots"
PLOTS_DIR.mkdir(exist_ok=True)

plt.rcParams.update({
    "figure.dpi": 110,
    "axes.titlesize": 16,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "axes.grid": True,
    "grid.alpha": 0.3,
})

COLORBLIND_FRIENDLY = ["#2E86AB", "#F18F01", "#A23B72", "#C73E1D", "#2EC4B6", "#33673B"]

if (PLOTS_DIR / "day02_solution_plot.png").exists():
    display(Image(filename=PLOTS_DIR / "day02_solution_plot.png", width=420))
else:
    print("Preview image not found; continue with the workflow.")


def load_data(csv_name: str, **read_kwargs) -> pd.DataFrame:
    path = DATA_DIR / csv_name
    df = pd.read_csv(path, **read_kwargs)
    print(f"Loaded {csv_name} → {df.shape[0]:,} rows × {df.shape[1]} columns.")
    return df


def validate_columns(df: pd.DataFrame, required: list[str]) -> None:
    missing = [col for col in required if col not in df.columns]
    if missing:
        warn(f"Missing columns: {missing}")
    else:
        print("✅ Columns look good:", required)


def expect_rows_between(df: pd.DataFrame, lower: int, upper: int) -> None:
    rows = len(df)
    if rows < lower or rows > upper:
        warn(f"Row count {rows:,} outside the expected range ({lower:,}–{upper:,}).")
    else:
        print(f"✅ Row count within expected range ({rows:,}).")


def quick_diagnostics(df: pd.DataFrame, name: str = "DataFrame") -> None:
    print(f"--- Quick check for {name} ---")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    print("Missing values:", df.isna().sum())
    display(df.head())


def check_story_fields(**fields) -> None:
    empty = [key for key, value in fields.items() if not str(value).strip()]
    if empty:
        warn(f"These storytelling fields still need text: {', '.join(empty)}")
    else:
        print("✅ Story scaffolding complete. Ready to plot!")


def baseline_style():
    plt.style.use("seaborn-v0_8-whitegrid")
    plt.rcParams["axes.facecolor"] = "#F8FAFC"


def save_last_fig(filename: str) -> None:
    path = PLOTS_DIR / filename
    plt.savefig(path, dpi=300, bbox_inches="tight")
    print(f"Figure saved to {path.relative_to(PROJECT_ROOT)}")


## Step 1. Load renewable share data
We'll start with the total renewable share dataset and sanity-check the structure.

In [ ]:
renewables_total = load_data("01 renewable-share-energy.csv")
quick_diagnostics(renewables_total, name="renewable share raw")


✅ **Checkpoint:** Expect columns `Entity`, `Code`, `Year`, and `Renewables (% equivalent primary energy)` covering ~300 entities.

## Step 2. Load component technology shares
Hydro, solar, and wind are provided as separate files. We'll bring them together so we can compare technologies side by side.

In [ ]:
hydro = load_data("06 hydro-share-energy.csv")
solar = load_data("14 solar-share-energy.csv")
wind = load_data("10 wind-share-energy.csv")

for name, frame in {"hydro": hydro, "solar": solar, "wind": wind}.items():
    validate_columns(frame, ["Entity", "Year"])
    expect_rows_between(frame, 2000, 10000)


In [ ]:
renewables_tidy = renewables_total.drop(columns=["Code"], errors="ignore")

def rename_value_column(df: pd.DataFrame, current: str, new: str) -> pd.DataFrame:
    subset = df.drop(columns=["Code"], errors="ignore").copy()
    return subset.rename(columns={current: new})

tech_shares = (
    rename_value_column(renewables_tidy, "Renewables (% equivalent primary energy)", "renewables_pct")
    .merge(
        rename_value_column(hydro, "Hydro (% equivalent primary energy)", "hydro_pct"),
        on=["Entity", "Year"],
        how="left"
    )
    .merge(
        rename_value_column(wind, "Wind (% equivalent primary energy)", "wind_pct"),
        on=["Entity", "Year"],
        how="left"
    )
    .merge(
        rename_value_column(solar, "Solar (% equivalent primary energy)", "solar_pct"),
        on=["Entity", "Year"],
        how="left"
    )
)

quick_diagnostics(tech_shares, name="merged technology shares")


### Mini self-diagnostic
If you see duplicate columns suffixed with `_x` or `_y`, double-check the `drop(columns=['Code'])` operations before the merges.

## Step 3. Focus on five key economies in 2023
We'll filter for the latest common year and compute an "other renewables" bucket to complete the stacked categories.

In [ ]:
focus_entities = ["World", "United States", "China", "India", "Germany"]
latest_year = tech_shares["Year"].max()
print(f"Latest year available: {latest_year}")

focus = (
    tech_shares[tech_shares["Entity"].isin(focus_entities)]
    .query("Year == @latest_year")
    [["Entity", "Year", "renewables_pct", "hydro_pct", "wind_pct", "solar_pct"]]
    .dropna(subset=["renewables_pct"])
)

for col in ["hydro_pct", "wind_pct", "solar_pct"]:
    focus[col] = focus[col].fillna(0)

focus["other_pct"] = (focus["renewables_pct"] - focus[["hydro_pct", "wind_pct", "solar_pct"]].sum(axis=1)).clip(lower=0)
focus = focus.sort_values("renewables_pct", ascending=False)

quick_diagnostics(focus, name="focus table")


✅ **Expectation check:** `other_pct` should be small but non-negative; if it's negative, revisit the subtraction logic.

## Step 4. Build the storytelling scaffold
Capture your main claim and supporting context before you render the grouped bars.

In [ ]:
TITLE = "Renewables top 20% of energy in Germany while lagging below 15% in the U.S."
SUBTITLE = f"Breakdown of renewable energy share by technology in {latest_year}"
ANNOTATION = "Wind dominates the U.S. renewable mix, whereas Germany leans on a more balanced portfolio with sizable solar."
SOURCE = "Source: Our World in Data – Energy Mix (BP Statistical Review)"
UNITS = "Units: Share of primary energy consumption (%)"

check_story_fields(
    TITLE=TITLE,
    SUBTITLE=SUBTITLE,
    ANNOTATION=ANNOTATION,
    SOURCE=SOURCE,
    UNITS=UNITS,
)


## Step 5. Plot grouped bars with an annotation pass
We'll create a grouped bar chart for each country, color the technologies using a colorblind-safe palette, and annotate the key contrast.

In [ ]:
baseline_style()
categories = ["hydro_pct", "wind_pct", "solar_pct", "other_pct"]
labels = ["Hydro", "Wind", "Solar", "Other"]
colors = COLORBLIND_FRIENDLY[:len(categories)]

fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.18
indices = range(len(focus))

for i, (cat, label, color) in enumerate(zip(categories, labels, colors)):
    offsets = [idx + i * bar_width for idx in indices]
    ax.bar(offsets, focus[cat], width=bar_width, label=label, color=color)

ax.set_xticks([idx + bar_width * (len(categories) - 1) / 2 for idx in indices])
ax.set_xticklabels(focus["Entity"], rotation=15, ha="right")
ax.set_ylabel(UNITS)

title_text = TITLE + "\n" + SUBTITLE
ax.set_title(title_text, loc="left", pad=18)
ax.legend(frameon=False, ncol=2)

max_country = focus.iloc[0]
ax.annotate(
    ANNOTATION,
    xy=(0 + bar_width, max_country["wind_pct"] + 2),
    xytext=(1.2, focus["renewables_pct"].max() + 5),
    arrowprops=dict(arrowstyle="->", color="#555"),
    fontsize=11,
    color="#333",
)

ax.set_ylim(0, max(30, focus["renewables_pct"].max() + 10))

caption_text = SOURCE + "\n" + "Claim → Evidence → Visual → Takeaway: Portfolios differ sharply; Germany combines technologies while the U.S. leans on wind alone."
fig.text(0.01, -0.08, caption_text, fontsize=10)
plt.show()


### Accessibility checklist
- Color palette tested for deuteranopia safety.
- Bars labeled via legend with text, not just color.
- Annotation arrow and text highlight the narrative contrast.
- Percent axis starts at zero to avoid exaggeration.

In [ ]:
save_last_fig("day02_solution_plot.png")


## Step 6. Reflect on teaching and uncertainty
- Shares depend on BP's accounting of primary energy; biofuels and waste may be grouped into "other".
- Country-level policy changes (feed-in tariffs, tax credits) explain rapid shifts but aren't captured in the data.
- Encourage students to explore a second year (e.g., 2000) to discuss change over time.
- Ask learners to note any surprising technology mixes and hypothesize the underlying drivers.